In [9]:
!pip install python-twitter

In [10]:
!pip install psycopg2-binary

In [11]:
import twitter

In [12]:
import json
import time
import psycopg2 as pg2
import numpy as np
import pandas as pd

from datetime import datetime
from psycopg2.extras import RealDictCursor, Json

In [13]:
%run sql_pw.py

In [14]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host=IP_ADDRESS, #allows you to navigate db
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    '''
    Executes a query directly to a database, without having to create a cursor and connection each time. 
    '''
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

In [15]:
twitter_keys = {
    'consumer_key':        'x2QffrlOw2v0oJjakKYIHsR6h',
    'consumer_secret':     'xViQ3VoIMUqWrdsXS4hTL0GuBVqPQx8lkxqxdibjEDLdinflnC',
    'access_token_key':    '319864871-2UQ33shLpkYA9ZcZfA7OXSYKrespIjwEZNS4tOHn',
    'access_token_secret': 'wpLbZp01SCcxscviBORW9bpTROCvmLe1PJ9006sjXdaQv'
}

api = twitter.Api(consumer_key         =   twitter_keys['consumer_key'],
                  consumer_secret      =   twitter_keys['consumer_secret'],
                  access_token_key     =   twitter_keys['access_token_key'],
                  access_token_secret  =   twitter_keys['access_token_secret'],
)

In [16]:
print(api.VerifyCredentials())

{"created_at": "Sat Jun 18 21:53:40 +0000 2011", "default_profile": true, "default_profile_image": true, "followers_count": 2, "friends_count": 2, "id": 319864871, "id_str": "319864871", "lang": "en", "name": "Mintaek Hong", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "MintaekH", "status": {"created_at": "Sun Mar 04 14:41:49 +0000 2012", "id": 176316523554684928, "id_str": "176316523554684928", "in_reply_to_screen_nam

In [17]:
def streamTweets(city, geocode, term, before, count, sql_db):
    for i in range(1,8):
        year, month, day = before.split('-')
        day = int(day)
        day-=1
        day = str(day).zfill(2)
        date = year + month + day
        after = datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
        
        results = api.GetSearch(
            term = term,
            geocode = geocode,
            return_json = True
        )

        insert_entry_json(results['statuses'], sql_db)
        before = after

In [18]:
# searchterms = ['man of the woods']

In [19]:
def tweet_repeater(city, geocode, term, start, repeats=15, count=100, sql_db='raw_tweets'):
    for i in range(repeats):
        before = start
        
        streamTweets(city, geocode, term, before, count, sql_db)
        print(f'Loop {i+1} complete. Raw tweets pushed to {sql_db}.')
        time.sleep(40)
        
    print('All tweets pulled.')

In [20]:
tweet_repeater(city='cha', 
               geocode='35.2271,-80.8431,10mi', 
               term = 'man of the woods',
               start='2019-01-08', 
               repeats=100, 
               count=100, 
               sql_db='raw_tweets')

Loop 1 complete. Raw tweets pushed to raw_tweets.


KeyboardInterrupt: 

In [21]:
query = """SELECT data->>'text'
FROM raw_tweets;
"""
response = execute_query(query, dict_cur=True)

df = pd.DataFrame(response).drop_duplicates()

In [22]:
df.columns = ['text']

In [24]:
df.reset_index(inplace=True)

In [25]:
df.drop('index', axis = 1, inplace = True)

In [26]:
df.tail()

,text
20694,@gulzaarAvi @kvnghakeem1 SDV
20695,"Says he's gonna teach me just what fast is, sa..."
20696,@ShimminyKricket Spot on nerdy! For those that...
20697,One of may fav shows on TV is back tonight. ht...
20698,and today i have a new job .. God come onnnnnn...


In [27]:
df.shape

(20699, 1)

In [28]:
import re

def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Eliminate hastags
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #Remove @ signs
    tweet = re.sub('@', '', tweet)
    return tweet

In [29]:
tweet_list = [i for i in df['text']]

In [30]:
tweet_list[:5]

['My article on Maori Davenport and the unreasonable ruling of Steve Savarese. Savarese’s comments are revealing, and… https://t.co/Klko7QAn0f',
 'Today’s AHSAA statement on Maori Davenport not only contains false and misleading assertions, it paints AHSAA as a… https://t.co/GDqhyeHbfQ',
 'Today’s AHSAA statement on Maori Davenport contains false and misleading assertions. The Illinois player DID deposi… https://t.co/k5l2cpULI8',
 'Power https://t.co/qcSkhvxfXO',
 '@amazon @AmazonHelp @USPS @USPSHelp - PLEASE you guys.  Again. Please deal with the manners of these delivery peopl… https://t.co/Y510McExtq']

Create a column of processed tweets utilizing the created function above:

In [31]:
df['processed'] = [processTweet(i) for i in tweet_list]

In [32]:
df.head()

,text,processed
0,My article on Maori Davenport and the unreason...,my article on maori davenport and the unreason...
1,Today’s AHSAA statement on Maori Davenport not...,today’s ahsaa statement on maori davenport not...
2,Today’s AHSAA statement on Maori Davenport con...,today’s ahsaa statement on maori davenport con...
3,Power https://t.co/qcSkhvxfXO,power URL
4,@amazon @AmazonHelp @USPS @USPSHelp - PLEASE y...,amazon amazonhelp usps uspshelp - please you g...


In [33]:
! pip install nltk

In [34]:
from nltk.tokenize import RegexpTokenizer

In [35]:
tokenizer = RegexpTokenizer(r'\w+')

In [36]:
df['clean_processed'] = df['processed'].map(lambda x: tokenizer.tokenize(x))

In [37]:
df.head()

,text,processed,clean_processed
0,My article on Maori Davenport and the unreason...,my article on maori davenport and the unreason...,"[my, article, on, maori, davenport, and, the, ..."
1,Today’s AHSAA statement on Maori Davenport not...,today’s ahsaa statement on maori davenport not...,"[today, s, ahsaa, statement, on, maori, davenp..."
2,Today’s AHSAA statement on Maori Davenport con...,today’s ahsaa statement on maori davenport con...,"[today, s, ahsaa, statement, on, maori, davenp..."
3,Power https://t.co/qcSkhvxfXO,power URL,"[power, URL]"
4,@amazon @AmazonHelp @USPS @USPSHelp - PLEASE y...,amazon amazonhelp usps uspshelp - please you g...,"[amazon, amazonhelp, usps, uspshelp, please, y..."


In [38]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [39]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
df['lemm_clean_processed'] = df['clean_processed'].map(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x]))

In [41]:
df.head()

,text,processed,clean_processed,lemm_clean_processed
0,My article on Maori Davenport and the unreason...,my article on maori davenport and the unreason...,"[my, article, on, maori, davenport, and, the, ...",my article on maori davenport and the unreason...
1,Today’s AHSAA statement on Maori Davenport not...,today’s ahsaa statement on maori davenport not...,"[today, s, ahsaa, statement, on, maori, davenp...",today s ahsaa statement on maori davenport not...
2,Today’s AHSAA statement on Maori Davenport con...,today’s ahsaa statement on maori davenport con...,"[today, s, ahsaa, statement, on, maori, davenp...",today s ahsaa statement on maori davenport con...
3,Power https://t.co/qcSkhvxfXO,power URL,"[power, URL]",power URL
4,@amazon @AmazonHelp @USPS @USPSHelp - PLEASE y...,amazon amazonhelp usps uspshelp - please you g...,"[amazon, amazonhelp, usps, uspshelp, please, y...",amazon amazonhelp usps uspshelp please you guy...


In [42]:
lemm_clean_processed_list = [i for i in df['lemm_clean_processed']]

In [43]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [44]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

Utilize countvectorizer:

min_df will remember words that are in a document at least once. For example, if min_df = 3, will only remember words that are a part of 3 documents in a corpus. max_df refers to the %-age of documents within a corpus that a word occurs in. For example, if max_df = 0.9, words that occur in more than 90% of my documents will be ignored. By default, max_df is equal to 1.

In [45]:
cv = CountVectorizer(ngram_range = (1,2),
                     stop_words = 'english',
                     min_df = 15,
                     max_df = 1.0)

In [60]:
df_cv = pd.SparseDataFrame(cv.fit_transform(df['lemm_clean_processed']), 
                     columns = cv.get_feature_names())

In [62]:
from sys import getsizeof

In [67]:
getsizeof(df_cv)

853152

In [66]:
df_cv.fillna(0, inplace=True)

In [68]:
df_cv.columns

Index(['00', '000', '01', '07', '10', '100', '11', '110', '110 free', '12',
       ...
       'yo', 'york', 'young', 'yourvoice', 'yourvoice america', 'youtube',
       'youtube playlist', 'youtube video', 'zero', 'zip'],
      dtype='object', length=1828)

Perform SVD to reduce dimensionality to about ~1000 (Currently ~1800)...Have to run an instance with about ~8GB of RAM (~0.08cents to perform one calculation); shut off the instance and restart this instance.. 

tfidf dataframe

In [79]:
tfidf = TfidfVectorizer(ngram_range=(1,2), 
                        stop_words = 'english', 
                        min_df = 25, 
                        max_df = 1.0)

In [80]:
df_tfidf = pd.SparseDataFrame(tfidf.fit_transform(df['lemm_clean_processed']),
                        columns = tfidf.get_feature_names())

In [81]:
df_tfidf.shape

(20699, 1019)

In [82]:
df_tfidf.fillna(0, inplace = True)

In [83]:
df_tfidf.head()

,00,000,10,100,11,12,120,13,130,130 free,...,year url,yep,yes,yesterday,yo,young,youtube,youtube playlist,youtube video,zip
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### HDBSCAN

In [94]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as data
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}

In [99]:
plt.figure(figsize=(20,10));
# plt.scatter(df_tfidf.T[0], df_tfidf.T[1])

<Figure size 1440x720 with 0 Axes>

In [100]:
! pip install hdbscan

OSError: [Errno 12] Cannot allocate memory

In [101]:
import hdbscan

ModuleNotFoundError: No module named 'hdbscan'

In [84]:
clusterer = hdbscan.HDBSCAN(min_cluster_size = 5, gen_min_span_tree = True)
cluster.fit(df_tfidf)

NameError: name 'hdbscan' is not defined

### SVD

In [87]:
# from sklearn.decomposition import TruncatedSVD
# from sklearn.metrics.pairwise import cosine_similarity

In [88]:
# SVD = TruncatedSVD(n_components=1000) 
# #doesn't center out data...versus PCA which it does...
# ##If we didn't fit before train_test_split (WHY DO WE NEED TO FIT TRANSFORM BEFORE TRAIN TEST SPLIT...)
# svd_matrix = SVD.fit_transform(df_tfidf)
# svd_matrix.shape

### Using spaCy

In [71]:
# ! pip install -U spacy

In [85]:
! python -m spacy download en_core_web_sm

/opt/conda/bin/python: No module named spacy


In [6]:
! pip install -U spacy

  Using cached https://files.pythonhosted.org/packages/ae/6e/a89da6b5c83f8811e46e3a9270c1aed90e9b9ee6c60faf52b7239e5d3d69/spacy-2.0.18-cp36-cp36m-manylinux1_x86_64.whl
Killed


In [92]:
# import spacy

In [3]:
# import spacy
# nlp = spacy.load('en_core_web_sm')
# punctuations = string.punctuation

# #Function to clean up text
# def cleanup_text(docs, logging=False):
#     texts = []
#     counter = 1
#     for doc in docs:
#         if counter % 1000 == 0 and logging:
#             print("Processed %d out of %d documents." % (counter, len(docs)))
#         counter += 1
#         doc = nlp(doc, disable=['parser', 'ner']) #disabling default models of spacy
#         tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']  #Applyint tokenizing and lemmatizing
#         tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations] #Removing stopwords and punctuations
#         tokens = ' '.join(tokens) 
#         texts.append(tokens)  #Appending clean titles to texts list
#     return pd.Series(texts)   #Returning texts list as pd Series

# tweets_clean= cleanup_text(lemm_clean_processed_list)
# tweets_clean = ' '.join(lemm_clean_processed_list).split()

# tweets_counts = Counter(tweets_clean)

# tweets_common_words = [word[0] for word in tweets_counts.most_common(20)]
# tweets_common_counts = [word[1] for word in tweets_counts.most_common(20)]
# fig = plt.figure(figsize=(18,6))
# sns.barplot(x=tweets_common_words, y=tweets_common_counts, palette = "GnBu_d")
# sns.set(font_scale=1.5, rc={'figure.figsize':(8, 8)})
# plt.title('Most Common Words used in tweets')
# plt.show()

ModuleNotFoundError: No module named 'spacy'